# Rough

In [206]:
import scipy.io as sio
import plotly.express as px
import numpy as np
from scipy.stats import median_abs_deviation
from scipy.signal import find_peaks, filtfilt, iirnotch, savgol_filter
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [542]:
data = sio.loadmat('training/chapman_shaoxing/g5/JS04193.mat')
fs = 500
leads = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
ecg = dict()
for lead, lead_data in zip(leads, data['val']):
  ecg[lead] = lead_data

In [543]:
signal = ecg['II']

In [545]:
# remove baseline wander and set median to 0
# https://python-heart-rate-analysis-toolkit.readthedocs.io/en/latest/_modules/heartpy/filtering.html
b, a = iirnotch(0.05 , Q = 0.005, fs = fs)
filtered_ecg = filtfilt(b, a, signal)
median = np.median(filtered_ecg)
filtered_ecg = filtered_ecg-median
fig = px.line(y=filtered_ecg, x=np.arange(len(filtered_ecg)), labels={'x':'t', 'y':'mV'})
mad = median_abs_deviation(filtered_ecg)
factor = 5
pos_deviation = factor*mad
fig.add_shape(type='line',
                x0=0,
                y0=0,
                x1=len(filtered_ecg),
                y1=0,
                line=dict(color='Red',),
                xref='x',
                yref='y'
)
fig.add_shape(type='line',
                x0=0,
                y0=pos_deviation,
                x1=len(filtered_ecg),
                y1=pos_deviation,
                line=dict(color='Green',),
                xref='x',
                yref='y'
)
fig.show()

In [546]:
# find Rs
R_indices, _ = find_peaks(filtered_ecg, height=factor*mad)
Rs = filtered_ecg[R_indices]
fig = make_subplots(shared_xaxes=True, shared_yaxes=True)
fig.add_trace(
    go.Line(y=filtered_ecg, x=np.arange(len(filtered_ecg)), name='ecg')
)
fig.add_trace(
    go.Scatter(x=R_indices, y=Rs, mode='markers', name='Rs')
)
fig.update_xaxes(title_text="t")
fig.update_yaxes(title_text="mV")
fig.show()

In [547]:
# fitler Rs
window = 20
true_R_indices = []
i = 0
while i<len(R_indices):
    cluster = [[Rs[i], R_indices[i]]]
    if i<len(R_indices)-1:
        while R_indices[i+1]-R_indices[i] < window:
            cluster.append([Rs[i+1], R_indices[i+1]])
            i = i+1
            if i == len(R_indices)-1:
                break
    true_R_indices.append(max(cluster)[1])
    i = i+1
true_R_indices = np.array(true_R_indices)

In [548]:
true_Rs = filtered_ecg[true_R_indices]
fig = make_subplots(shared_xaxes=True, shared_yaxes=True)
fig.add_trace(
    go.Line(y=filtered_ecg, x=np.arange(len(filtered_ecg)), name='ecg')
)
fig.add_trace(
    go.Scatter(x=true_R_indices, y=true_Rs, mode='markers', name='Rs')
)
fig.update_xaxes(title_text="t")
fig.update_yaxes(title_text="mV")
fig.show()

In [251]:
# cut ecg
margin = 100
cut_ecg = filtered_ecg[true_R_indices[1]+margin:true_R_indices[-2]-margin]
cut_true_R_indices = true_R_indices[true_R_indices > true_R_indices[1]+margin]
cut_true_R_indices = cut_true_R_indices[cut_true_R_indices < true_R_indices[-2]-margin]
cut_true_R_indices = cut_true_R_indices - (true_R_indices[1]+margin)
cut_true_Rs = cut_ecg[cut_true_R_indices]
S_factor = 12
S_cutoff = -S_factor*mad
fig = make_subplots(shared_xaxes=True, shared_yaxes=True)
fig.add_trace(
    go.Line(y=cut_ecg, x=np.arange(len(cut_ecg)), name='ecg')
)
fig.add_trace(
    go.Scatter(x=cut_true_R_indices, y=cut_true_Rs, mode='markers', name='Rs')
)
fig.add_trace(
    go.Scatter(x=[0,len(cut_ecg)], y=[S_cutoff, S_cutoff], mode='lines', name='S cutoff')
)
fig.update_xaxes(title_text="t")
fig.update_yaxes(title_text="mV")
fig.show()

In [549]:
# average RR interval (R to R)
rr_intervals = []
for i in range(len(cut_true_R_indices)-1):
    rr_intervals.append(cut_true_R_indices[i+1] - cut_true_R_indices[i])
avg_rr_interval = np.mean(rr_intervals)
print('average RR interval (in seconds):', avg_rr_interval * 1/fs)

average RR interval (in seconds): 0.39009090909090904


In [559]:
# find S peaks
S_indices, _ = find_peaks(-1*cut_ecg, height=S_factor*mad, distance=100)
true_S_indices = S_indices
true_Ss = cut_ecg[S_indices]
# Ss = -cut_ecg[S_indices]
# # fitler Ss
# window = 100
# true_S_indices = []
# i = 0
# while i<len(S_indices):
#     cluster = [[Ss[i], S_indices[i]]]
#     if i<len(S_indices)-1:
#         while S_indices[i+1]-S_indices[i] < window:
#             cluster.append([Ss[i+1], S_indices[i+1]])
#             i = i+1
#             if i == len(S_indices)-1:
#                 break
#     true_S_indices.append(max(cluster)[1])
#     i = i+1
# true_S_indices = np.array(true_S_indices)
# true_Ss = cut_ecg[true_S_indices]


In [562]:
Q_factor = 5
Q_cutoff = -Q_factor*mad
fig = make_subplots(shared_xaxes=True, shared_yaxes=True)
fig.add_trace(
    go.Line(x=np.arange(len(cut_ecg)), y=-cut_ecg, name='ecg')
)
# fig.add_trace(
#     go.Scatter(x=cut_true_R_indices, y=-cut_true_Rs, mode='markers', name='Rs')
# )
fig.add_trace(
    go.Scatter(x=true_S_indices, y=-true_Ss, mode='markers', name='Ss')
)
fig.add_trace(
    go.Scatter(x=[0,len(cut_ecg)], y=[-S_cutoff, -S_cutoff], mode='lines', name='S cutoff')
)
# fig.add_trace(
#     go.Scatter(x=[0,len(cut_ecg)], y=[-Q_cutoff, -Q_cutoff], mode='lines', name='Q cutoff')
# )
fig.update_xaxes(title_text="t")
fig.update_yaxes(title_text="mV")
fig.show()

In [275]:
Q_indices = _find_peaks(cut_ecg, min_height=-Q_cutoff, max_height=-S_cutoff, reverse=True)
Qs = cut_ecg[Q_indices]
true_Q_indices = filter_peaks(-Qs, Q_indices, window=50)
true_Qs = cut_ecg[true_Q_indices]
fig = make_subplots(shared_xaxes=True, shared_yaxes=True)
fig.add_trace(
    go.Line(y=cut_ecg, x=np.arange(len(cut_ecg)), name='ecg')
)
fig.add_trace(
    go.Scatter(x=cut_true_R_indices, y=cut_true_Rs, mode='markers', name='Rs')
)
fig.add_trace(
    go.Scatter(x=true_S_indices, y=true_Ss, mode='markers', name='Ss')
)
fig.add_trace(
    go.Scatter(x=true_Q_indices, y=true_Qs, mode='markers', name='Qs')
)
fig.update_xaxes(title_text="t")
fig.update_yaxes(title_text="mV")
fig.show()

In [263]:
Q_indices

array([ 140,  144,  433,  436,  441,  645,  976,  979, 1269, 1273, 1275,
       1509, 1513, 1518, 1815, 1819, 2026, 2030, 2307, 2310, 2522, 2527,
       2829, 2833, 3070, 3091, 3286, 3604, 3607])

# Final Code

In [309]:
import numpy as np

import scipy.io as sio
from scipy.stats import median_abs_deviation
from scipy.signal import find_peaks, filtfilt, iirnotch, savgol_filter

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import typing

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [363]:
def filter_peaks(peaks:np.ndarray, peak_indices:np.ndarray, window:int, inverted:bool=False) -> np.ndarray:
    if inverted:
        peaks = -1*peaks
    true_peak_indices = []
    i = 0
    while i<len(peak_indices):
        cluster = [[peaks[i], peak_indices[i]]]
        if i<len(peak_indices)-1:
            while peak_indices[i+1]-peak_indices[i] < window:
                cluster.append([peaks[i+1], peak_indices[i+1]])
                i = i+1
                if i == len(peak_indices)-1:
                    break
        true_peak_indices.append(max(cluster)[1])
        i = i+1
    true_peak_indices = np.array(true_peak_indices)
    return true_peak_indices

In [316]:
def plot_ecg(graphs:list, title:str=None):
    fig = make_subplots(shared_xaxes=True, shared_yaxes=True)
    for graph in graphs:
        fig.add_trace(graph)
    fig.update_xaxes(title_text="t")
    fig.update_yaxes(title_text="mV")
    if title:
        fig.update_layout(title_text=title)
    return fig

In [347]:
def _find_peaks(ecg:np.ndarray, min_height:float, max_height:float=None, inverted:bool=False) -> np.ndarray:
    sign = 1
    if inverted:
        sign = -1
    if max_height:
        height = (sign*min_height, sign*max_height)
    else:
        height = sign*min_height
    peak_indices, _ = find_peaks(sign*ecg, height=height)
    return peak_indices

In [459]:
def load_data(file_path:str) -> dict:
    data = sio.loadmat(file_path)
    leads = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    ecgs = dict()
    for lead, lead_data in zip(leads, data['val']):
      ecgs[lead] = lead_data
    return ecgs

In [386]:
def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]

In [519]:
ecgs = load_data('training/chapman_shaoxing/g5/JS04193.mat')
fs = 500
signal = ecgs['II']

In [520]:
# plot signal
fig = px.line(x=np.arange(len(signal)), y=signal, labels={'x':'t', 'y':'mV'}, title='original ecg')
fig.show()

In [521]:
# remove baseline wander and set median to 0
# https://python-heart-rate-analysis-toolkit.readthedocs.io/en/latest/_modules/heartpy/filtering.html
b, a = iirnotch(0.05 , Q = 0.005, fs = fs)
bw_fixed_ecg = filtfilt(b, a, signal)
median = np.median(bw_fixed_ecg)
bw_fixed_ecg = bw_fixed_ecg-median
bw_fixed_ecg_graph = go.Line(y=bw_fixed_ecg, x=np.arange(len(bw_fixed_ecg)), name='ecg')

In [522]:
fig = plot_ecg([bw_fixed_ecg_graph], title='fixed ecg')
fig.show()

In [523]:
# median absolute deviation (kind of like standard deviation but from the median rather than the mean)
mad = median_abs_deviation(bw_fixed_ecg)

In [526]:
# deciding the cutoff for R
R_factor = 5
R_cutoff = R_factor*mad
R_cutoff_graph = go.Scatter(x=[0,len(bw_fixed_ecg)], y=[R_cutoff, R_cutoff], mode='lines', name='R cutoff')

# finding and filtering Rs
R_indices = _find_peaks(bw_fixed_ecg, min_height=R_cutoff)
Rs = bw_fixed_ecg[R_indices]
true_R_indices = filter_peaks(Rs, R_indices, window=100)
true_Rs = bw_fixed_ecg[true_R_indices]
Rs_graph = go.Scatter(x=true_R_indices, y=true_Rs, mode='markers', name='Rs')

In [527]:
fig = plot_ecg([bw_fixed_ecg_graph, R_cutoff_graph, Rs_graph], title='marked')
fig.show()

In [528]:
# cut signal according to Rs
margin = 100
cut_ecg = bw_fixed_ecg[true_R_indices[1]+margin:true_R_indices[-2]-margin]

cut_ecg_graph = go.Line(y=cut_ecg, x=np.arange(len(cut_ecg)), name='ecg')

# fix Rs because of shift in x-axis
cut_true_R_indices = true_R_indices[true_R_indices > true_R_indices[1]+margin]
cut_true_R_indices = cut_true_R_indices[cut_true_R_indices < true_R_indices[-2]-margin]
true_R_indices = cut_true_R_indices - (true_R_indices[1]+margin)
true_Rs = cut_ecg[true_R_indices]

# fix graphs
Rs_graph = go.Scatter(x=true_R_indices, y=true_Rs, mode='markers', name='Rs')
R_cutoff_graph = go.Scatter(x=[0,len(cut_ecg)], y=[R_cutoff, R_cutoff], mode='lines', name='R cutoff')

In [529]:
fig = plot_ecg([cut_ecg_graph, R_cutoff_graph, Rs_graph], title='cut ecg')
fig.show()

In [540]:
# deciding the cutoff for S
S_factor = 4
S_cutoff = -S_factor*mad
S_cutoff_graph = go.Scatter(x=[0,len(cut_ecg)], y=[S_cutoff, S_cutoff], mode='lines', name='S cutoff')

# finding and filtering Ss
S_indices = _find_peaks(cut_ecg, min_height=S_cutoff, inverted=True)
Ss = cut_ecg[S_indices]
true_S_indices = filter_peaks(Ss, S_indices, window=100, inverted=True)
true_Ss = cut_ecg[true_S_indices]
Ss_graph = go.Scatter(x=true_S_indices, y=true_Ss, mode='markers', name='Ss')

In [541]:
fig = plot_ecg([cut_ecg_graph, R_cutoff_graph, Rs_graph, S_cutoff_graph, Ss_graph], title='marked ecg')
fig.show()

In [ ]:
# update S cutoff to the min

In [449]:
# deciding the cutoff for Q
Q_factor = 4
Q_cutoff = -Q_factor*mad
Q_cutoff_graph = go.Scatter(x=[0,len(cut_ecg)], y=[Q_cutoff, Q_cutoff], mode='lines', name='Q cutoff')

# finding and filtering Qs
Q_indices = _find_peaks(cut_ecg, min_height=Q_cutoff, max_height=S_cutoff, inverted=True)
Qs = cut_ecg[Q_indices]
true_Q_indices = filter_peaks(Qs, Q_indices, window=50, inverted=True)
true_Qs = cut_ecg[true_Q_indices]
Qs_graph = go.Scatter(x=true_Q_indices, y=true_Qs, mode='markers', name='Qs')

In [450]:
fig = plot_ecg([cut_ecg_graph, R_cutoff_graph, Rs_graph, S_cutoff_graph, Ss_graph, Q_cutoff_graph, Qs_graph], title='marked ecg')
fig.show()

In [451]:
# average RR interval (R to R)
rr_intervals = []
for i in range(len(true_R_indices)-1):
    rr_intervals.append(true_R_indices[i+1] - true_R_indices[i])
rr_intervals = np.array(rr_intervals)
rr_intervals = reject_outliers(rr_intervals, 3)
avg_rr_interval = np.mean(rr_intervals)
print('average RR interval (in seconds):', avg_rr_interval * 1/fs)
print('variance in RR intervals (in seconds):', np.var(rr_intervals) * 1/fs)

average RR interval (in seconds): 0.5329230769230769
variance in RR intervals (in seconds): 4.609727810650888


In [452]:
# average QRS interval

# find first zero crossing before Q index
Q_zero_crossings = []
for Q_index in true_Q_indices:
    i = Q_index
    flag = 1
    while cut_ecg[i] < 0:
        i = i - 1
        if i < 0:
            flag = 0
            break
    if flag:
        Q_zero_crossings.append(i)
Q_zero_crossings = np.array(Q_zero_crossings)

Q_zero_crossings_graph = go.Scatter(x=Q_zero_crossings, y=cut_ecg[Q_zero_crossings], mode='markers', name='Q zero crossings')

# find first zero crossing after S index
S_zero_crossings = []
for S_index in true_S_indices:
    i = S_index
    flag = 1
    while cut_ecg[i] < 0:
        i = i + 1
        if i > len(cut_ecg)-2:
            flag = 0
            break
    if flag:
        S_zero_crossings.append(i)
S_zero_crossings = np.array(S_zero_crossings)

S_zero_crossings_graph = go.Scatter(x=S_zero_crossings, y=cut_ecg[S_zero_crossings], mode='markers', name='S zero crossings')


In [453]:
fig = plot_ecg([cut_ecg_graph, R_cutoff_graph, Rs_graph, S_cutoff_graph, Ss_graph, Q_cutoff_graph, Qs_graph, Q_zero_crossings_graph, S_zero_crossings_graph], title='marked ecg')
fig.show()

In [454]:
# average QRS interval
qrs_intervals = S_zero_crossings - Q_zero_crossings
qrs_intervals = reject_outliers(qrs_intervals, 3)
avg_qrs_interval = np.mean(qrs_intervals)
print('average QRS interval (in seconds):', avg_qrs_interval * 1/fs)
print('variance in QRS intervals (in seconds):', np.var(qrs_intervals) * 1/fs)

average QRS interval (in seconds): 0.08815384615384617
variance in QRS intervals (in seconds): 0.03337278106508875


# Filters

In [ ]:
ecgs = load_data('training/chapman_shaoxing/g3/JS02081.mat')
fs = 500
signal = ecgs['II']

In [ ]:
# plot signal
fig = px.line(x=np.arange(len(signal)), y=signal, labels={'x':'t', 'y':'mV'}, title='original ecg')
fig.show()

In [ ]:
# remove baseline wander and set median to 0
# https://python-heart-rate-analysis-toolkit.readthedocs.io/en/latest/_modules/heartpy/filtering.html
b, a = iirnotch(0.05 , Q = 0.005, fs = fs)
bw_fixed_ecg = filtfilt(b, a, signal)
median = np.median(bw_fixed_ecg)
bw_fixed_ecg = bw_fixed_ecg-median
bw_fixed_ecg_graph = go.Line(y=bw_fixed_ecg, x=np.arange(len(bw_fixed_ecg)), name='ecg')

In [383]:
# plot bw_fixed_ecg
fig = px.line(x=np.arange(len(bw_fixed_ecg)), y=bw_fixed_ecg, labels={'x':'t', 'y':'mV'}, title='fixed ecg')
fig.show()

In [382]:
smoothened_ecg = savgol_filter(bw_fixed_ecg, 36, 5)
fig = px.line(y=smoothened_ecg, x=np.arange(len(smoothened_ecg)), labels={'x':'t', 'y':'mV'}, title='smooth ecg')
fig.show()

In [384]:
def moving_average(ecg, window):
    return np.convolve(ecg, np.ones(window), 'valid') / window

averaged_ecg = moving_average(bw_fixed_ecg, window=4)

In [385]:
fig = px.line(y=averaged_ecg, x=np.arange(len(averaged_ecg)), labels={'x':'t', 'y':'mV'}, title='averaged ecg')
fig.show()